In [1]:
import os
import numpy as np
from tqdm import tqdm
import yaml
import onnxruntime as ort
import numpy as np

# PyTorch and torchvision imports
import torch
from torch.utils.data import DataLoader 
import torchvision.transforms as T

# Custom models and datasets
from dataset.video_dataset import AdImageSequenceDataset
from models.features_extractors import EfficientNet_feature_B5

# Custom utils
from utils.utils import CosineLoss, GaussianSmoothing

# Sklearn for evaluation
from sklearn.metrics import roc_auc_score

import yaml
import torch

# Clear CUDA cache and enable CuDNN for better performance
torch.cuda.empty_cache()
torch.backends.cudnn.enabled = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.8/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load configuration from YAML file
with open('configs/config_inference.yaml', 'r') as file:
    config = yaml.safe_load(file)


In [3]:
# Define image preprocessing transformations for EfficientNet
preprocessing_efficientnet = T.Compose([
    T.Resize((config['general_config']['image_size'], 
              config['general_config']['image_size']), 
             interpolation=T.InterpolationMode.BICUBIC),
    T.ToTensor(),
    T.Normalize(mean=[0.4850, 0.4560, 0.4060],
                std=[0.2290, 0.2240, 0.2250])
])

# Define image preprocessing transformations for labels
preprocessing_labels = T.Compose([
    T.Resize((config['general_config']['feature_size'], config['general_config']['feature_size'])),
    T.ToTensor()
])

# Create validation dataset
dataset_val = AdImageSequenceDataset(
    os.path.join(config['general_config']['data_path']),
    preprocessing_efficientnet,
    preprocessing_labels
)

dataloader_val = DataLoader(
    dataset_val,
    batch_size=1,
    shuffle=False,
    num_workers=6
)


gs_filter = GaussianSmoothing()

In [4]:
model_SN = torch.load(config['general_config']['weight_path_SimpleNet']).to(device)
model_SN.eval()

model_GeneralAD = torch.load(config['general_config']['weight_path_generalAD']).to(device)
model_GeneralAD.eval()


model_PatchCore = torch.load(config['general_config']['weight_path_PatchCore'])

model_VMTAD = torch.load(config['general_config']['weight_path_VMTAD']).to(device)
model_VMTAD.mode = 'stream'
model_VMTAD.eval()

onnx_model_path = config['general_config']['weight_path_VMTAD_onnx']
VMTAD_session = ort.InferenceSession(onnx_model_path, providers=['CUDAExecutionProvider'])

feature_extraction  = EfficientNet_feature_B5(config)
feature_extraction.eval() 

print("Models load")

<ipython-input-4-33794d1d1548>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_SN = torch.load(config['general_config']['weight_path_SimpleNet']).to(device)
<ipython-

Models load


## Vanilla Version of VMTAD

The vanilla version of VMTAD is used for AUROC results and inference time.

In [5]:
# Evaluation setup 
eval_seg = np.zeros((len(dataset_val),config['general_config']['feature_size'],config['general_config']['feature_size']))
labels_seg = np.zeros((len(dataset_val),config['general_config']['feature_size'],config['general_config']['feature_size']))
eval_det = np.zeros(len(dataset_val))
labels_det = np.zeros(len(dataset_val))

# Loss setup
loss_visu = CosineLoss()

# tqdm setup
t = tqdm(total = len(dataloader_val))

# loop setup
folder_indice = None

# evaluation loop
for k,(_, folder, images, labels) in enumerate(dataloader_val):

    images_cuda = images.to('cuda')
    labels_np = labels[0,1].numpy().astype(int)
    with torch.no_grad():
        i  = 1
        if folder_indice != folder :
            folder_indice = folder
            i = 0
            
        outputs = model_VMTAD(feature_extraction(images_cuda),i)
        pred_frames = outputs['reconstructed_frames']
        original_frame = feature_extraction(images_cuda)
        
        anomaly_map = loss_visu(pred_frames,original_frame)
        anomaly_map_smooth = gs_filter(anomaly_map)[0].detach().cpu().numpy()
        
        eval_seg[k] = anomaly_map_smooth
        labels_seg[k] = labels_np
                
        eval_det[k] = np.max(anomaly_map_smooth)
        labels_det[k] = np.max(labels_np)
        
        t.update()
        t.set_description_str(str(folder))
        
# Print evaluation results        
print("AUROC Seg: ",roc_auc_score(labels_seg.flatten(),eval_seg.flatten()))
print("AUROC Det: ",roc_auc_score(labels_det,eval_det))

('AD_5',): 100%|██████████| 2990/2990 [05:34<00:00,  9.05it/s]

AUROC Seg:  0.9976342667864593
AUROC Det:  0.9598744282823254


## ONNX Version

ONNX model conversion degrades the AUROC performance. This version is open-weight.

In [6]:

# Evaluation setup 
eval_seg = np.zeros((len(dataset_val),config['general_config']['feature_size'],config['general_config']['feature_size']))
labels_seg = np.zeros((len(dataset_val),config['general_config']['feature_size'],config['general_config']['feature_size']))
eval_det = np.zeros(len(dataset_val))
labels_det = np.zeros(len(dataset_val))

# Loss setup
loss_visu = CosineLoss()

# tqdm setup
t = tqdm(total = len(dataloader_val))

# loop setup
folder_indice = None

# evaluation loop
for k,(_, folder, images, labels) in enumerate(dataloader_val):

    images_cuda = images.to('cuda')
    labels_np = labels[0,1].numpy().astype(int)
    with torch.no_grad():
        i  = 1
        if folder_indice != folder :
            folder_indice = folder
            i = 0
            
        onnx_input_name = VMTAD_session.get_inputs()[0].name  
        input_features=feature_extraction(images_cuda).detach().cpu().numpy().astype(np.float32)
        outputs = VMTAD_session.run(['output', 'src'], {onnx_input_name: input_features})
        output = outputs[0]
        src = outputs[1]
        visu_loss = loss_visu(torch.from_numpy(output).to('cuda'),torch.from_numpy(src).to('cuda'))
        anomaly_map_smooth = gs_filter(visu_loss).detach().cpu().numpy()
        
        eval_seg[k] = anomaly_map_smooth
        labels_seg[k] = labels_np
                
        eval_det[k] = np.max(anomaly_map_smooth)
        labels_det[k] = np.max(labels_np)
        
        t.update()
        t.set_description_str(str(folder))
        
# Print evaluation results        
print("AUROC Seg: ",roc_auc_score(labels_seg.flatten(),eval_seg.flatten()))
print("AUROC Det: ",roc_auc_score(labels_det,eval_det))

('AD_5',): 100%|██████████| 2990/2990 [06:49<00:00,  7.31it/s]


AUROC Seg:  0.9973894177983017
AUROC Det:  0.9557240241127287


# SimpleNet

In [5]:
# Evaluation setup 
eval_seg = np.zeros((len(dataset_val),config['general_config']['feature_size'],config['general_config']['feature_size']))
labels_seg = np.zeros((len(dataset_val),config['general_config']['feature_size'],config['general_config']['feature_size']))
eval_det = np.zeros(len(dataset_val))
labels_det = np.zeros(len(dataset_val))

# tqdm setup
t = tqdm(total = len(dataloader_val))

# evaluation loop
for k,(_, folder, images, labels) in enumerate(dataloader_val):

    images_cuda = images.to('cuda')
    labels_np = labels[0,1].numpy().astype(int)
    with torch.no_grad():
            
        outputs = model_SN(feature_extraction(images_cuda), mode='inference')
        anomaly_map_smooth = gs_filter(outputs)[0].detach().cpu().numpy()
        
        eval_seg[k] = anomaly_map_smooth
        labels_seg[k] = labels_np
                
        eval_det[k] = np.max(anomaly_map_smooth)
        labels_det[k] = np.max(labels_np)
        
        t.update()
        t.set_description_str(str(folder))
        
# Print evaluation results        
print("AUROC Seg: ",roc_auc_score(labels_seg.flatten(),eval_seg.flatten()))
print("AUROC Det: ",roc_auc_score(labels_det,eval_det))

('AD_5',): 100%|██████████| 2990/2990 [00:56<00:00, 53.49it/s]

AUROC Seg:  0.9910839844262312
AUROC Det:  0.9445148340463093


('AD_5',): 100%|██████████| 2990/2990 [01:10<00:00, 53.49it/s]

# GeneralAD

In [6]:
# Evaluation setup 
eval_seg = np.zeros((len(dataset_val),config['general_config']['feature_size'],config['general_config']['feature_size']))
labels_seg = np.zeros((len(dataset_val),config['general_config']['feature_size'],config['general_config']['feature_size']))
eval_det = np.zeros(len(dataset_val))
labels_det = np.zeros(len(dataset_val))

# tqdm setup
t = tqdm(total = len(dataloader_val))

# evaluation loop
for k,(_, folder, images, labels) in enumerate(dataloader_val):

    images_cuda = images.to('cuda')
    labels_np = labels[0,1].numpy().astype(int)
    with torch.no_grad():
            
        outputs = model_GeneralAD(feature_extraction(images_cuda), generate_anomaly=False)
        anomaly_map_smooth = gs_filter(outputs).detach().cpu().numpy()
        
        eval_seg[k] = anomaly_map_smooth
        labels_seg[k] = labels_np
                
        eval_det[k] = np.max(anomaly_map_smooth)
        labels_det[k] = np.max(labels_np)
        
        t.update()
        t.set_description_str(str(folder))
        
# Print evaluation results        
print("AUROC Seg: ",roc_auc_score(labels_seg.flatten(),eval_seg.flatten()))
print("AUROC Det: ",roc_auc_score(labels_det,eval_det))

('AD_5',): 100%|██████████| 2990/2990 [01:51<00:00, 26.77it/s]


AUROC Seg:  0.997777891531634
AUROC Det:  0.9533309442519351


# PatchCore

In [7]:
# Define image preprocessing transformations for labels
preprocessing_labels = T.Compose([
    T.Resize((config['general_config']['feature_size']-2, config['general_config']['feature_size']-2)),
    T.ToTensor()
])

# Create validation dataset
dataset_val = AdImageSequenceDataset(
    os.path.join(config['general_config']['data_path']),
    preprocessing_efficientnet,
    preprocessing_labels
)


dataloader_val = DataLoader(
    dataset_val,
    batch_size=1,
    shuffle=False
)


# Evaluation setup 
eval_seg = np.zeros((len(dataset_val),config['general_config']['feature_size']-2,config['general_config']['feature_size']-2))
labels_seg = np.zeros((len(dataset_val),config['general_config']['feature_size']-2,config['general_config']['feature_size']-2))
eval_det = np.zeros(len(dataset_val))
labels_det = np.zeros(len(dataset_val))

t = tqdm(total = len(dataloader_val))

# evaluation loop
for k,(_, folder, images, labels) in enumerate(dataloader_val):

    images_cuda = images.to('cuda')
    labels_np = labels[0,1].numpy().astype(int)
    with torch.no_grad():
            
        outputs = model_PatchCore.compute_anomaly_map(feature_extraction(images_cuda))
        anomaly_map_smooth = gs_filter(outputs[1].unsqueeze(0)).detach().cpu().numpy()
        eval_seg[k] = anomaly_map_smooth
        labels_seg[k] = labels_np
                
        eval_det[k] = np.max(anomaly_map_smooth)
        labels_det[k] = np.max(labels_np)
        
        t.update()
        t.set_description_str(str(folder))
        
# Print evaluation results        
print("AUROC Seg: ",roc_auc_score(labels_seg.flatten(),eval_seg.flatten()))
print("AUROC Det: ",roc_auc_score(labels_det,eval_det))

('AD_5',): 100%|██████████| 2990/2990 [10:00<00:00,  4.98it/s]


('AD_5',): 100%|██████████| 2990/2990 [05:33<00:00,  9.23it/s]

AUROC Seg:  0.9747343690569894
AUROC Det:  0.9494130653266333
